# class drake::geometry::QueryObject< T >

In [1]:
# Import some basic libraries and functions for this tutorial.
import numpy as np
import os

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, SnoptSolver, IpoptSolver,  
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint
    )

In [2]:
h = 1e-3
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=h)
parser = Parser(plant)

In [3]:
dir1 = "/Users/xiao/0_codes/ICBM_drake/models/objects"
dir2 = "/Users/xiao/0_codes/ICBM_drake/models/ycb/sdf/"
table_file = os.path.join(dir1, "table_top.sdf")
panda_file = "/Users/xiao/0_codes/ICBM_drake/models/franka_description/urdf/panda_arm_hand.urdf"
box_file = os.path.join(dir2, "003_cracker_box.sdf")
table_modelInstance = parser.AddModels(table_file)[0]  # this return pydrake.multibody.tree.ModelInstanceIndex
panda_modelInstance = parser.AddModels(panda_file)[0]
box_modelInstance = parser.AddModels(box_file)[0]

table_top_frame = plant.GetFrameByName("table_top_center")
robot_base = plant.GetFrameByName("panda_link0")

X_W_table = RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],]),
  p=[0.0, 0.0, 0.0]
)

plant.WeldFrames(plant.world_frame(), table_top_frame, X_W_table)
plant.WeldFrames(table_top_frame, robot_base, RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.]))
plant.Finalize()
plant.set_name("table_with_box")
diagram = builder.Build()
context = diagram.CreateDefaultContext()
mutable_context = plant.GetMyContextFromRoot(context)

<div class="alert alert-block alert-success">
<b>Querying the scene geometry based on context at $t_k$:</b> 1. create mutable_contexts from root_context for each time step; 2. at $t_k$, using its corresponding mutable_context to create a query_port; 3. create query_object, which has some useful member functions; 4. create the inspector = query_object.inspector() to extract informations from the scene geometries.
</div>

In [5]:
query_port = plant.get_geometry_query_input_port()

In [9]:
if query_port.HasValue(mutable_context):
    query_object = query_port.Eval(mutable_context)

In [10]:
signed_distance_pairs = query_object.ComputeSignedDistancePairwiseClosestPoints()

In [26]:
print(len(signed_distance_pairs))

2165


In [16]:
print(signed_distance_pairs[0].id_A)
print(signed_distance_pairs[0].id_B)

<GeometryId value=21>
<GeometryId value=282>


In [23]:
# using the inspector to get info on the pairs
inspector = query_object.inspector()
frameA_id = inspector.GetFrameId(signed_distance_pairs[0].id_A)
frameB_id = inspector.GetFrameId(signed_distance_pairs[1].id_B)

bodyA = plant.GetBodyFromFrameId(frameA_id)
bodyB = plant.GetBodyFromFrameId(frameB_id)

In [24]:
type(bodyA)

pydrake.multibody.tree.RigidBody

In [25]:
print(bodyA)
print(bodyB)

<RigidBody name='table_top_link' index=1 model_instance=2>
<RigidBody name='base_link_cracker' index=14 model_instance=4>
